In [ ]:
#Librerias y parametros de redes neuronales
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras import backend as ff

#Librerias de matriz de confusion
from keras.preprocessing.image import ImageDataGenerator
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np

#Librerias principales
import tensorflowjs
import os

ff.clear_session() 

In [ ]:
#Parametros basicos
shape       = (256, 256, 3)
pasos_train = 120          
dim         = (256, 256)
batch_size  = 32              #32
pasos       = 400             #400
epocas      = 30              #30    
dataset1    = "./train/"
dataset2    = "./test/"
destino     = "./ultimo v3/"

In [ ]:
#Generar datasets

gen  = ImageDataGenerator(
    rescale=1. /255,                
    horizontal_flip=True,           
    vertical_flip=True,              
    zoom_range=[1, 1.5],            
    brightness_range=(0.1, 0.9))     

gen2  = ImageDataGenerator(rescale=1. /255)
gen3  = ImageDataGenerator(rescale=1./ 255)

train_ds = gen.flow_from_directory(
    directory=dataset1, 
    target_size=dim,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

test_ds = gen2.flow_from_directory(
    directory=dataset2,
    target_size=dim,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)                 

matriz_ds = gen3.flow_from_directory(
    directory="./matriz dataset/",
    target_size=dim, 
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)    

num_clases = train_ds.num_classes
print(train_ds.class_indices)

In [ ]:
#Tipo de red neuronal convolucional, convoliciones y pooling
dcnn = Sequential()
dcnn.add(Conv2D(input_shape = shape, filters=64, kernel_size=(3,3), activation='relu', padding="same"))
dcnn.add(MaxPooling2D(pool_size=(2,2)))
dcnn.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding="same"))
dcnn.add(MaxPooling2D(pool_size=(2,2)))
dcnn.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding="same"))
dcnn.add(MaxPooling2D(pool_size=(2,2)))

#Arquitectura de redes
dcnn.add(Flatten()) 
dcnn.add(Dense(256, activation='relu')) 
dcnn.add(Dropout(0.5))
dcnn.add(Dense(128, activation="relu"))  
dcnn.add(Dropout(0.5))                                 
dcnn.add(Dense(3, activation='softmax')) 
print(dcnn.summary())
dcnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [ ]:
#Entrenamiento de CNN
red_neuronal = dcnn.fit(train_ds, epochs=epocas, steps_per_epoch=pasos,
    validation_data=test_ds, validation_steps=pasos_train, verbose=1, workers=6)

In [ ]:
#Guardado en H5
#Nota: Tratar de automatizar mas eso de las carpetas, y solo cambiar 1 direccion
if not os.path.exists(destino):
    os.mkdir(destino)

modelo_dir = destino + 'Modelo.h5'
pesos_dir  = destino + 'Pesos neuronales.h5'
dcnn.save(modelo_dir)
dcnn.save_weights(pesos_dir)

#Guardado de JS
carpeta2 = destino + 'modelo java/'
if not os.path.exists(carpeta2):
    os.mkdir(carpeta2)
    
tensorflowjs.converters.save_keras_model(dcnn, carpeta2)
print("modelos guardado con exito")

In [ ]:
#Funcion perdida y precision por epoca
import matplotlib.pyplot as plt
plt.xlabel("Epoca")
plt.ylabel("Loss / Accuracy")
plt.plot(red_neuronal.history["loss"])
plt.plot(red_neuronal.history["categorical_accuracy"])
plt.savefig(destino + "Funcion de perdida y precision por epocas.jpg")

In [ ]:
#Matriz de confusion real 1: Cargar datos, y modelo
#Recordar usar el dataset de test

#Nombres de las categorias
etiquetas    = ["Excelente", "Bajo-Medio", "Alto-Grave"]

#Cargar modelo y cargar dataset test
modelo       = load_model(modelo_dir) #Aqui ir cambiando de modleo
predicciones = modelo.predict_generator(generator=test_ds)

#Ejes 
y_prediccion = np.argmax(predicciones, axis=1)
y_real       = test_ds.classes

In [ ]:
#Generar matriz de confusion
matriz = confusion_matrix(y_real, y_prediccion)
plot_confusion_matrix(conf_mat=matriz, figsize=(4,4), class_names=etiquetas, show_normed=False)
plt.tight_layout()
plt.savefig(destino + "matriz.jpg")

In [ ]:
#Matriz de confusion 2: Medida de precision
#Sacar foto
print(metrics.classification_report(y_real, y_prediccion, digits=4))

In [ ]:
#Matriz 2: Con el mismos datos del dataset 
predicciones2 = modelo.predict_generator(generator=matriz_ds)

#Generar matriz de confusion
y_real2 = matriz_ds.classes
y_prediccion2 = np.argmax(predicciones2, axis=1)

matriz2 = confusion_matrix(y_real2, y_prediccion2)
plot_confusion_matrix(conf_mat=matriz2, figsize=(4,4), class_names=etiquetas, show_normed=False)
plt.tight_layout()
plt.savefig(destino + "matriz2.jpg")

In [ ]:
print(metrics.classification_report(y_real2, y_prediccion2, digits=4))